In [2]:
import glob
import os
from os.path import join
from natsort import natsorted
import rasterio
import imageio.v2 as imageio
import numpy as np
from tqdm import tqdm
import cv2
import time
import copy

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim 
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
from torch.utils.data import TensorDataset, DataLoader, random_split

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import colors

colors_list = ['yellow', 'pink', 'teal', 'gray', 'red', 'white', 'aqua', 'orange', 'darkblue']
cmap = mpl.colors.ListedColormap(colors_list)
bounds = [-0.5, 0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5, 256]
norm = colors.BoundaryNorm(bounds, cmap.N)

print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)


PyTorch Version:  1.14.0.dev20221103+cu117
Torchvision Version:  0.15.0.dev20221103+cu117


In [17]:
# import random
# data = imageio.imread('/mnt/ceph/onehealth/Boyu/land_cover/land_cover_features/bafodia_lc_fea_1000m_res_10m.tif')
# print(data.dtype)
# sum(data[random.randint(0,1000), random.randint(0,2000), :])

float32


IndexError: index 647 is out of bounds for axis 0 with size 432

In [3]:
model_path = '/mnt/ceph/boyuz/Builiding_Landscaping/land_conver_segmentation/partial_label/code/cnn_model_5_epoch_30.pt'

raster_list = [
    '/mnt/ceph/boyuz/BU_Classifier/Bafodia_Google.tif'
    ]

pred_list = [
    '/mnt/ceph/boyuz/Builiding_Landscaping/land_conver_segmentation/partial_label/code/64_5_Bafodia_Google.npy'
]

tile_size = 64
step = 5 
site_id = 1
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
PIXEL = 0.2204315

radius = 25
resolution = 1 if radius < 200 else 10
factor = PIXEL/resolution
site = raster_list[site_id].split(os.sep)[-1][:-4]
print(f'Currently processing {site}, radius {str(radius)}')

Currently processing Bafodia, radius 25


In [14]:
# get original raster meta
with rasterio.open(raster_list[site_id]) as src:
    data = src.read()
    profile = src.profile
    transform = src.transform
    height, width = src.height, src.width
    crs = src.crs

# create new raster_meta
new_height, new_width = int(height * factor), int(width * factor)

new_transform = transform * transform.scale(
                    (width / new_width),
                    (height / new_height)
                )
new_profile = profile

new_profile.update(transform=new_transform, driver='GTiff', height=new_height, width=new_width, crs=crs, count=8)

src.close()

In [ ]:
# load prediction of CNN model
predictions = np.load(pred_list[site_id])

In [ ]:
# pad the edges with repeat num_lines times
num_lines = len(list(range(0, 32, 5))) - 1

first_row = predictions[0,:]
repeated_first_row = np.repeat(np.expand_dims(first_row, 0), num_lines, axis=0)

last_row = predictions[-1,:]
repeated_last_row = np.repeat(np.expand_dims(last_row, 0), num_lines, axis=0)

new_predictions = np.concatenate([repeated_first_row, predictions, repeated_last_row], axis=0)

#expand vertically
first_col = new_predictions[:, 0]
repeated_first_col = np.repeat(np.expand_dims(first_col, 1), num_lines, axis=1)

last_col = new_predictions[:, -1]
repeated_last_col = np.repeat(np.expand_dims(last_col,1), num_lines, axis=1)

new_predictions = np.concatenate([repeated_first_col, new_predictions, repeated_last_col], axis=1)
print(new_predictions.shape)


(6172, 7783)


In [ ]:
# resize the padded label to original raster size
resized_preds = cv2.resize(new_predictions, (height, width), interpolation = cv2.INTER_NEAREST)

(6172, 7783) (30864, 38915)


In [ ]:
# create a cycle kernel based on distance
k_size = int(round(radius/PIXEL))
weights = np.zeros((k_size*2+1, k_size*2+1))
num_kernal_pixel = 0
for i in range(weights.shape[0]):
    for j in range(weights.shape[1]):
        if ((i-k_size)**2 + (j-k_size)**2)**0.5 <= k_size:
            weights[i, j] = 1
            num_kernal_pixel += 1
print(num_kernal_pixel)

40089


In [ ]:
# calculate convolution for each class
pixel_maps = []
for class_idx in range(8):
    print(f'Calculating category {str(class_idx)}...')
    conv_target = copy.copy(resized_preds)
    if class_idx == 0:
        conv_target[conv_target==0] = 255
        conv_target[conv_target!=255] = 0
        conv_target[conv_target==255] = 1
    else:
        conv_target[conv_target!=class_idx] = 0
        conv_target[conv_target==class_idx] = 1
    
    # calculate the building number with GPU
    lbl_full_tensor = torch.from_numpy(conv_target).unsqueeze(0).unsqueeze(0)
    # lbl_full_tensor = lbl_full_tensor
    weights_tensor = torch.from_numpy(weights)
    weights_tensor = weights_tensor.view(1, 1, weights_tensor.shape[0], weights_tensor.shape[1]).repeat(1, 1, 1, 1)
    pixel_counts_idx = torch.nn.functional.conv2d(lbl_full_tensor.float().to(device), weights_tensor.float().to(device), padding='same')
    pixel_counts_idx = np.squeeze(pixel_counts_idx.cpu().numpy()) #get back from gpu
    pixel_counts_idx = pixel_counts_idx/num_kernal_pixel # get percentage

    resized_pixel_counts= cv2.resize(pixel_counts_idx, (new_width, new_height))
    pixel_maps.append(pixel_counts_idx)
    break
# save the results as a npy file
np.save(f'lc_feature_{str(radius)}m_res_{str(resolution)}m', pixel_maps)

Calculating category 0...


In [21]:
# load the downsampled fc_feature
pixel_maps = np.load(f'{site.lower()}_lc_fea_{str(radius)}m_res_{str(resolution)}m.npy')
print(len(pixel_maps))

8


In [24]:
# save the new data according to the created profile
with rasterio.open(f'{site.lower()}_lc_fea_{str(radius)}m_res_{str(resolution)}m.tif', 'w', **new_profile) as dst:
    for i in range(len(pixel_maps)):
        dst.write(pixel_maps[i].astype(float), i+1)
dst.close()